In [1]:
import numpy as np
import jax.numpy as jnp
import jaxopt
from jax import jit
from exojax.chem.mass_action import logK_FC

In [3]:
cc='''
(2.1)
H2 <-> 2 H + 0 C + 0 O
CO <-> 0 H + 1 C + 1 O
CH4 <-> 4 H + 1 C + 0 O
H2O <-> 2 H + 0 C + 1 O
C2H2 <-> 2 H + 2 C + 0 O

(2.2)
n_H2   = K_H2* x_H^2* x_C^0* x_O^0
n_CO   = K_CO*  x_H^0* x_C^1* x_O^1
n_CH4  = K_CH4* x_H^4* x_C^1* x_O^0 
n_H2O  = K_H2O* x_H^2* x_C^0* x_O^1 
n_C2H2  = K_C2H2* x_H^2* x_C^2* x_O^0 

(2.3)
ep_H xbH = xH + 4 n_CH4 + 2 n_H2O
ep_C xbH = xC + n_CO + n_CH4
ep_O xbH = xO + n_CO + n_H2O
'''

In [106]:
#from FastChem
#logK = a1/T + a2 ln T + a3 + a4 T + a5 T^2 for FastChem:
#H2 Hydrogen : H 2 # Chase, M. et al., JANAF thermochemical tables, 1998.
kpH2=[5.1909637142380554e+04,-1.8011701211306956e+00,8.7224583233705744e-02,2.5613890164973008e-04,-5.3540255367406060e-09]
#C1O1 Carbon_Monoxide : C 1 O 1 # Chase, M. et al., JANAF thermochemical tables, 1998.
kpCO=[1.2899777785630804e+05,-1.7549835812545211e+00,-3.1625806804795502e+00,4.1336204683783961e-04 ,-2.3579962985989574e-08] 
#C1H4 Methane : C 1 H 4 # Chase, M. et al., JANAF thermochemical tables, 1998.
kpCH4=[1.9784584536781305e+05,-8.8316803072239054e+00,5.2793066855988400e+00,2.7567674752936866e-03,-1.3966691995535711e-07]
#H2O1 Water : H 2 O 1 # Chase, M. et al., JANAF thermochemical tables, 1998.
kpH2O=[1.1033645388793820e+05,-4.1783597409582285e+00,3.1744691010633233e+00,9.4064684023068001e-04,-4.0482461482866891e-08]
#C2H2 Ethyne : C 2 H 2 # Chase, M. et al., JANAF thermochemical tables, 1998.
kpC2H2=[1.9646042660345597e+05,-4.5142919982618537e+00,-1.2427863494177018e+01,1.5177662181888874e-03,7.4542139989784635e-08]

In [107]:
ma_coeff_=jnp.array([kpH2,kpCO,kpCH4,kpH2O,kpC2H2]).T

In [108]:
from exojax.atm.idealgas import number_density
T=1500.0
nh_=number_density(1.0,T)

epsilon_=np.array([0.84,12*4012/739000,16*10400/739000.])
logK_=logK_FC(T,nuf,ma_coeff_)

In [8]:
logK_

DeviceArray([-21.12764 ,  27.547245, -95.67498 , -38.54748 , -41.08718 ],            dtype=float32)

In [9]:
b_=epsilon_*nh_
b_

array([4.05606490e+18, 3.14574374e+17, 1.08726271e+18])

In [109]:
nuf

array([[ 2., nan, nan],
       [nan,  1.,  1.],
       [ 4.,  1., nan],
       [ 2., nan,  1.],
       [ 2.,  2., nan]], dtype=float32)

In [124]:
nuf=np.array([[2,0,0],[0,1,1],[4,1,0],[2,0,1],[2,2,0]],dtype=np.float32) #(2.1) formula matrix

def zero_replaced_nuf(nuf):
    """calc zero-replaced formula matrix
    Args:
        nuf: formula matrix
        
    Returns:
        zero-replaced formula matrix (float32) 
    """
    znuf=np.copy(nuf)
    znuf[znuf==0]=np.nan
    return znuf

def calc_nufmask(nuf):
    """calc zero-replaced to nan formula matrix mask
    Args:
        nuf: formula matrix
        
    Returns:
        nufmask (float32) 
    """
    nufmask=np.copy(nuf)
    msk=nufmask==0
    nufmask[~msk]=1.0
    nufmask[msk]=np.nan
    return nufmask


def calc_Nj(nuf):
    """calc Nj (2.25) in Stock et al.(2018)
    Args:
        nuf: formula matrix
        
    Returns:
        Nj
        
    """
    znuf=zero_replaced_nuf(nuf)
    return np.nanmax(znuf,axis=0)

def calc_epsiloni(nufmask,epsilon):
    """calc tilde(epsilon)=epsilon_i (2.24) in Stock et al.(2018)
    
    Args:
        nufmask: formula matrix mask
        epsilon: element abundance
        
    Returns:
        tilde(epsilon) = epsilon_i
    
    """
    emat=(jnp.full_like(nufmask,1)*epsilon)
    return jnp.nanmin(emat*nufmask,axis=1)
    

In [119]:

calc_nufmask(nuf)

array([[ 1., nan, nan],
       [nan,  1.,  1.],
       [ 1.,  1., nan],
       [ 1., nan,  1.],
       [ 1.,  1., nan]], dtype=float32)

In [120]:
nuf,epsilon_

(array([[2., 0., 0.],
        [0., 1., 1.],
        [4., 1., 0.],
        [2., 0., 1.],
        [2., 2., 0.]], dtype=float32),
 array([0.84      , 0.0651475 , 0.22516915]))

In [125]:
znuf=zero_replaced_nuf(nuf)
nufmask=calc_nufmask(nuf)
calc_epsiloni(nufmask,epsilon_)

DeviceArray([0.84      , 0.0651475 , 0.0651475 , 0.22516915, 0.0651475 ],            dtype=float32)

In [126]:
calc_Nj(nuf)

array([4., 2., 1.], dtype=float32)